# Crawling PTA dan Berita & Preprocessing

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re, sys, time

## 1. Crawling PTA

In [2]:
BASE_URL = "https://pta.trunojoyo.ac.id/c_search/byprod"

In [3]:
def get_max_page(prodi_id):
    url = f"{BASE_URL}/{prodi_id}/1"
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "html.parser")

    # Cari tombol >> (last page)
    last_page = soup.select_one('ol.pagination a:contains("»")')
    if last_page and "href" in last_page.attrs:
        href = last_page["href"]
        # Pecah URL -> ambil angka terakhir
        max_page = int(href.split("/")[-1])
        return max_page

    # fallback kalau pagination tidak ada
    return 1

In [4]:
# Contoh pemakaian
print(get_max_page(10))

172


/home/codespace/.local/lib/python3.12/site-packages/soupsieve/css_parser.py:876: FutureWarning: The pseudo class ':contains' is deprecated, ':-soup-contains' should be used moving forward.
  warnings.warn(  # noqa: B028


In [6]:
def print_progress(prodi_id, prodi, current_page, total_pages):
    percent = (current_page / total_pages) * 100
    bar_length = 20
    filled_length = int(bar_length * current_page // total_pages)
    bar = '█' * filled_length + '-' * (bar_length - filled_length)
    sys.stdout.write(f'\r[{prodi_id}] {prodi} - Page {current_page}/{total_pages} [{bar}] {percent:.2f}%')
    sys.stdout.flush()
    if current_page == total_pages:
        sys.stdout.write('\n')

## Crawling Data Fakultas Teknik

In [7]:
def pta_teknik():
    start_time = time.time()

    data = {
        "id": [],
        "penulis": [],
        "judul": [],
        "abstrak_id": [],
        "abstrak_en": [],
        "pembimbing_pertama": [],
        "pembimbing_kedua": [],
        "prodi": []
    }

    # daftar prodi yang akan diproses
    prodi_list = [9, 10, 11, 19, 20, 23, 31, 32, 33]
    total_pages = 0
    max_pages_dict = {}

    # hitung total halaman (untuk tiap prodi yang dipilih)
    for i in prodi_list:
        max_page = get_max_page(i)
        max_pages_dict[i] = max_page
        total_pages += max_page

    # scraping data tiap prodi
    for i in prodi_list:
        max_page = max_pages_dict[i]
        for j in range(1, max_page + 1):
            url = f"{BASE_URL}/{i}/{j}"
            r = requests.get(url)
            soup = BeautifulSoup(r.content, "html.parser")
            jurnals = soup.select('li[data-cat="#luxury"]')

            isii = soup.select_one('div#begin')
            if not isii:
                continue
            prodi_full = isii.select_one('h2').text.strip()
            prodi = prodi_full.replace("Journal Jurusan ", "")

            for jurnal in jurnals:
                link_keluar = jurnal.select_one('a.gray.button')['href']

                # ambil ID dari link PTA
                id_match = re.search(r"/detail/(\d+)", link_keluar)
                pta_id = id_match.group(1) if id_match else None

                response = requests.get(link_keluar)
                soup1 = BeautifulSoup(response.content, "html.parser")
                isi = soup1.select_one('div#content_journal')

                judul = isi.select_one('a.title').text.strip()
                penulis = isi.select_one('span:contains("Penulis")').text.split(' : ')[1]
                pembimbing_pertama = isi.select_one('span:contains("Dosen Pembimbing I")').text.split(' : ')[1]
                pembimbing_kedua = isi.select_one('span:contains("Dosen Pembimbing II")').text.split(' :')[1]

                paragraf = isi.select('p[align="justify"]')
                abstrak_id = paragraf[0].get_text(strip=True) if len(paragraf) > 0 else "N/A"
                abstrak_en = paragraf[1].get_text(strip=True) if len(paragraf) > 1 else "N/A"

                data["id"].append(pta_id)
                data["penulis"].append(penulis)
                data["judul"].append(judul)
                data["abstrak_id"].append(abstrak_id)
                data["abstrak_en"].append(abstrak_en)
                data["pembimbing_pertama"].append(pembimbing_pertama)
                data["pembimbing_kedua"].append(pembimbing_kedua)
                data["prodi"].append(prodi)

            # update progress bar per prodi
            print_progress(i, prodi, j, max_page)

        sys.stdout.write("\n")  # pindah baris setelah 1 prodi selesai

    # simpan ke CSV
    df = pd.DataFrame(data)
    df.to_csv("pta_teknik.csv", index=False, encoding="utf-8-sig")

    # hitung durasi
    end_time = time.time()
    elapsed = int(end_time - start_time)
    jam, sisa = divmod(elapsed, 3600)
    menit, detik = divmod(sisa, 60)

    # summary
    print("\n✅ Seluruh data berhasil dikumpulkan!")
    print(f"📊 Total entri: {len(df)}")
    print(f"⏱ Waktu eksekusi: {jam} jam {menit} menit {detik} detik")

    return df

In [8]:
pta_teknik()

[9] Teknik Industri - Page 143/143 [████████████████████] 100.00%

[10] Teknik Informatika - Page 172/172 [████████████████████] 100.00%

[11] Manajemen Informatika - Page 56/56 [████████████████████] 100.00%

[19] Teknik Multimedia Dan Jaringan - Page 27/27 [████████████████████] 100.00%

[20] Mekatronika - Page 28/28 [████████████████████] 100.00%

[23] Teknik Elektro - Page 34/34 [████████████████████] 100.00%

[31] Sistem Informasi - Page 1/1 [████████████████████] 100.00%

[32] Teknik Mesin - Page 1/1 [████████████████████] 100.00%

[33] Teknik Mekatronika - Page 1/1 [████████████████████] 100.00%


✅ Seluruh data berhasil dikumpulkan!
📊 Total entri: 2289
⏱ Waktu eksekusi: 0 jam 34 menit 56 detik


,id,penulis,judul,abstrak_id,abstrak_en,pembimbing_pertama,pembimbing_kedua,prodi
0,080421100005,Siliwangi Fitra Rachmawanto S.T.,OPTIMASI PEMILIHAN PORTOFOLIO SAHAM PERUSAHAAN...,Portofolio adalah sekumpulan saham yang dimili...,Portofolio is a collection of stock owned by i...,"Heri Awalul Ilhamsah S.T., M.T.","Retno Indriartiningtias S.T., M.T.",Teknik Industri
1,080421100087,AHMAD MAS'UD,PERANCANGAN TATA LETAK FASILITAS LANTAI PRODUK...,PT. ABC merupakan perusahaan yang bergerak dib...,PT. ABC is a company engaged in the manufactur...,"SABARUDIN AKHMAD, S.T., M.T.","SUGENG PURWOKO, S.T., M.T.",Teknik Industri
2,080421100019,Yulianto Fauzanta,PERUMUSAN STRATEGI BISNIS UD. BUDI JAYA BANGKA...,Bangkalan merupakan salah satu kabupaten yang ...,Bangkalan is one of the districts that have th...,"Fitri Agustina, S.T., M.T","Retno Indriartiningtias, S.T., M.T",Teknik Industri
3,080421100055,M Mundir Muhlisin,USULAN PERBAIKAN UTILITAS RESOURCES PADA LANTA...,Simulasi adalah duplikasi atau abstraksi dari ...,Simulation is a duplication or abstraction of ...,Mu'alim ST MT,Sugeng Purwoko ST MT,Teknik Industri
4,080421100046,Muhibbin,Peningkatan Kepuasan Masyarakat Terhadap Pelay...,Kepuasan adalah tingkat perasaan seseorang ter...,Satisfaction is feeling level of someone to se...,Rahmad Hidayat,Retno Indriartiningtias,Teknik Industri
...,...,...,...,...,...,...,...,...
2284,160491200026,Mochamad Izar Bahroni,Kestabilan Autonomous Quadcopter Dengan Kontro...,Unmanned Aerial Vehicle (UAV) memiliki banyak ...,"Unmanned Aerial Vehicle (UAV) has many uses, i...","Sri Wahyuni, S.Kom., M.T","Faikul Umam, S.Kom., M.T",Teknik Mekatronika
2285,160491200023,Moch Zaini,perancangan dan pembuatan sistem rem elektrik ...,Kendaraan merupakan suatu alat yang banyak dig...,Vehicle is a tool that is widely used by human...,"Hairil Budiarto., ST.MT","Faikul Umam., S.Kom.,MT",Teknik Mekatronika
2286,160491200027,Mohammad Syafruddin,Pengatur Kerapatan Kertas Pada Alat Penggulung...,PT PAKERIN adalah salah satu pabrik kertas ter...,PT PAKERIN is one of the largest paper mills i...,"Faikul Umam S.Kom., M.T","Ahmad Sahru Romadhon S.Kom., M.T",Teknik Mekatronika
2287,160491200028,"Khanif Khoirul Umam, A.Md.",IMPLEMENTASI ALGORITMA MAZE MAPPING PENCARIAN ...,Robot shortest path merupakan jenis robot yang...,Shortest path robot is a type of robot that is...,"Sri Wahyuni, S.Kom., M.T.","Hairil Budiarto, S.T.,M.T.",Teknik Mekatronika


## Link Keluaran Fakultas Teknik

In [9]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urljoin, urlparse
import urllib3

# matikan warning SSL insecure
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

def scrape_all_links(base_url, max_pages=50):
    visited = set()
    results = []

    def scrape_page(url):
        try:
            response = requests.get(url, verify=False, timeout=10)
            soup = BeautifulSoup(response.text, 'html.parser')

            # ambil semua link keluar
            for a in soup.find_all('a', href=True):
                href = a['href'].strip()
                if not href or href == "#":
                    continue
                full_link = urljoin(url, href)
                results.append({
                    "Page": url,
                    "Link Keluar": full_link
                })

            # cari link internal untuk dilanjutkan
            for a in soup.find_all('a', href=True):
                href = a['href'].strip()
                if not href or href == "#":
                    continue
                full_link = urljoin(url, href)
                # hanya ambil link internal (masih di domain utama)
                if full_link.startswith(base_url) and full_link not in visited:
                    visited.add(full_link)
                    if len(visited) < max_pages:
                        scrape_page(full_link)

        except Exception as e:
            print(f"⚠ Gagal akses {url}: {e}")

    # mulai dari base_url
    visited.add(base_url)
    scrape_page(base_url)

    # rapikan dataframe
    df = pd.DataFrame(results).reset_index(drop=True)
    df.index += 1
    df.insert(0, "No", df.index)
    return df

# contoh penggunaan
url = "https://informatika.trunojoyo.ac.id/"
df_links = scrape_all_links(url, max_pages=30)  # max_pages = batas biar ga infinite loop

df_links
# df_links.to_csv("semua_link.csv", index=False, encoding="utf-8-sig")

,No,Page,Link Keluar
1,1,https://informatika.trunojoyo.ac.id/,https://informatika.trunojoyo.ac.id/
2,2,https://informatika.trunojoyo.ac.id/,https://informatika.trunojoyo.ac.id/bidang-min...
3,3,https://informatika.trunojoyo.ac.id/,https://informatika.trunojoyo.ac.id/bidang-min...
4,4,https://informatika.trunojoyo.ac.id/,https://informatika.trunojoyo.ac.id/bidang-min...
5,5,https://informatika.trunojoyo.ac.id/,https://informatika.trunojoyo.ac.id/bidang-min...
...,...,...,...
1649,1649,https://informatika.trunojoyo.ac.id/unit-kegia...,https://tofatek.wordpress.com/
1650,1650,https://informatika.trunojoyo.ac.id/unit-kegia...,https://informatika.trunojoyo.ac.id/berita-pro...
1651,1651,https://informatika.trunojoyo.ac.id/unit-kegia...,https://informatika.trunojoyo.ac.id/berita-pro...
1652,1652,https://informatika.trunojoyo.ac.id/unit-kegia...,https://informatika.trunojoyo.ac.id/berita-pro...


## 2. Crawling Berita

### Fungsi

In [10]:
# Fungsi progress bar manual
def print_progress(kategori, current_page, total_pages):
    percent = (current_page / total_pages) * 100
    bar_length = 20
    filled_length = int(bar_length * current_page // total_pages)
    bar = '█' * filled_length + '-' * (bar_length - filled_length)
    sys.stdout.write(f'\r{kategori} - Page {current_page}/{total_pages} [{bar}] {percent:.2f}%')
    sys.stdout.flush()
    if current_page == total_pages:
        sys.stdout.write('\n\n')

In [11]:
# Fungsi untuk ambil konten artikel
def get_article_content(url):
    r = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
    soup = BeautifulSoup(r.text, "html.parser")

    paragraphs = []
    content_divs = soup.find_all("div", id="content-wrapper")
    for div in content_divs:
        for p in div.find_all("p"):
            text = p.get_text(strip=True)
            if text and not text.lower().startswith("baca juga"):
                paragraphs.append(text)
    return " ".join(paragraphs)

### Crawling Berita

In [12]:
def berita(categories, pages_per_category=1):
    start_time = time.time()  # mulai hitung waktu

    BASE_URL = "https://www.tempo.co/indeks?page={}&category=rubrik&rubric_slug={}"

    data = {
        "id_berita": [],
        "judul_berita": [],
        "isi_berita": [],
        "kategori_berita": []
    }

    for cat_id, cat in enumerate(categories, start=1):
        for page in range(1, pages_per_category+1):
            url = BASE_URL.format(page, cat)
            r = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
            soup = BeautifulSoup(r.text, "html.parser")

            articles = soup.select("figure figcaption a")
            for a in articles:
                link = "https://www.tempo.co" + a["href"]
                title = a.get_text(strip=True)

                id_match = re.search(r"-(\d+)$", link)
                berita_id = id_match.group(1) if id_match else None

                try:
                    content = get_article_content(link)
                except:
                    content = ""

                data["id_berita"].append(berita_id)
                data["judul_berita"].append(title)
                data["isi_berita"].append(content)
                data["kategori_berita"].append(cat)

            print_progress(cat, page, pages_per_category)

    df = pd.DataFrame(data)
    df.to_csv("tempo_berita.csv", index=False, encoding="utf-8-sig")

    end_time = time.time()
    elapsed = int(end_time - start_time)
    jam, sisa = divmod(elapsed, 3600)
    menit, detik = divmod(sisa, 60)

    # summary
    print("\n✅ Seluruh data berhasil dikumpulkan!")
    print(f"📊 Total entri: {len(df)}")
    print(f"⏱️ Waktu eksekusi: {jam} jam {menit} menit {detik} detik")

    return df

categories = ["politik", "hukum", "ekonomi", "hiburan", "internasional", "otomotif", "olahraga"]

In [5]:
berita(categories, pages_per_category=5)

politik - Page 5/5 [████████████████████] 100.00%

hukum - Page 5/5 [████████████████████] 100.00%

ekonomi - Page 5/5 [████████████████████] 100.00%

hiburan - Page 5/5 [████████████████████] 100.00%

internasional - Page 5/5 [████████████████████] 100.00%

otomotif - Page 5/5 [████████████████████] 100.00%

olahraga - Page 5/5 [████████████████████] 100.00%


✅ Seluruh data berhasil dikumpulkan!
📊 Total entri: 100
⏱️ Waktu eksekusi: 0 jam 0 menit 54 detik


,id_berita,judul_berita,isi_berita,kategori_berita
0,2074547,Mayoritas Dapur MBG di Bandung Masih Proses Se...,KEPALA Dinas Kesehatan Kota Bandung Sony Adam ...,politik
1,2074538,Kelakar Prabowo ke Dedi Mulyadi: Kalau Berengs...,PRESIDENPrabowoSubianto sempat berkelakar kepa...,politik
2,2074525,"Bangunan Pesantren Al Khoziny Sidoarjo Ambruk,...",BANGUNAN Asrama Putra PondokPesantrenAl Khozin...,politik
3,2074513,Politikus NasDem Singgung Perombakan Pejabat BGN,ANGGOTA Komisi IX Dewan Perwakilan Rakyat Irma...,politik
4,2074503,Cerita Prabowo Disindir Trump soal Pidatonya d...,PRESIDENPrabowoSubianto mengenang momen saat i...,politik
...,...,...,...,...
95,2073930,Arwani Tegaskan Tidak Ada Cawe-cawe dalam Mukt...,,politik
96,2073928,Sekjen PPP: Muktamar X Bakal Bahas Upaya Perba...,,politik
97,2073926,Prabowo akan Panggil Kepala BGN Bahas Keracuna...,,politik
98,2073921,Kemenag Luncurkan Modul Kesehatan Reproduksi R...,,politik


### Page & Link Keluar Berita

In [24]:
def berita_links(categories, pages_per_category=1):
    start_time = time.time()  # mulai hitung waktu

    BASE_URL = "https://www.tempo.co/indeks?page={}&category=rubrik&rubric_slug={}"

    data = {
        "id_berita": [],
        "page": [],
        "link_keluar": []
    }

    for cat in categories:
        for page in range(1, pages_per_category+1):
            url = BASE_URL.format(page, cat)
            r = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
            soup = BeautifulSoup(r.text, "html.parser")

            articles = soup.select("figure figcaption a")
            for a in articles:
                link = "https://www.tempo.co" + a["href"]
                id_match = re.search(r"-(\d+)$", link)
                berita_id = id_match.group(1) if id_match else None

                data["id_berita"].append(berita_id)
                data["page"].append(url)         # link page
                data["link_keluar"].append(link) # link detail

            # update progress bar
            print_progress(cat, page, pages_per_category)

    df = pd.DataFrame(data)
    df.to_csv("tempo_links.csv", index=False, encoding="utf-8-sig")

    end_time = time.time()
    elapsed = int(end_time - start_time)
    jam, sisa = divmod(elapsed, 3600)
    menit, detik = divmod(sisa, 60)

    # summary
    print("\n✅ Seluruh link berhasil dikumpulkan!")
    print(f"📊 Total entri: {len(df)}")
    print(f"⏱️ Waktu eksekusi: {jam} jam {menit} menit {detik} detik")

    return df

categories = ["politik", "hukum", "ekonomi", "hiburan", "internasional", "otomotif", "olahraga"]

In [25]:
berita_links(categories, pages_per_category=5)

politik - Page 1/5 [████----------------] 20.00%

politik - Page 5/5 [████████████████████] 100.00%

hukum - Page 5/5 [████████████████████] 100.00%

ekonomi - Page 5/5 [████████████████████] 100.00%

hiburan - Page 5/5 [████████████████████] 100.00%

internasional - Page 5/5 [████████████████████] 100.00%

otomotif - Page 5/5 [████████████████████] 100.00%

olahraga - Page 5/5 [████████████████████] 100.00%


✅ Seluruh link berhasil dikumpulkan!
📊 Total entri: 0
⏱️ Waktu eksekusi: 0 jam 0 menit 1 detik


,id_berita,page,link_keluar


## 3. Pre-Pocessing PTA dan Berita

### Library

In [26]:
!pip install pandas nltk spacy Sastrawi pyspellchecker
!python -m spacy download en_core_web_sm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.9/33.9 MB 50.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 869.3/869.3 kB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 44.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.0/802.0 kB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31/31 [spacy]m30/31 [spacy]tion]ta]

[notice] A new release of pip is available: 25.1

### Pre-Pocessing PTA

In [ ]:
import pandas as pd
import re
import nltk
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.corpus import stopwords
from spellchecker import SpellChecker
import spacy

# Download stopwords (sekali saja)
nltk.download('stopwords')

# Load spaCy English model
nlp = spacy.load("en_core_web_sm")

# === Load dataset ===
pta_all = pd.read_csv("pta_teknik.csv")

# === Daftar Prodi Fakultas Teknik ===
prodi_teknik = [
    "Teknik Industri",
    "Teknik Informatika",
    "Manajemen Informatika",
    "Teknik Multimedia Dan Jaringan",
    "Mekatronika",
    "Teknik Elektro",
    "Sistem Informasi",
    "Teknik Mesin",
    "Teknik Mekatronika"
]

# === Filter hanya Fakultas Teknik ===
pta_teknik = pta_all[pta_all["prodi"].isin(prodi_teknik)].copy()

# === Stopwords ===
stopwords_id = set(stopwords.words("indonesian"))
stopwords_en = set(stopwords.words("english"))

# === Stemmer Indonesia ===
factory = StemmerFactory()
stemmer_id = factory.create_stemmer()

# === Spell checker English ===
spell_en = SpellChecker(language="en")

# ========================
# Fungsi Preprocessing Indo
# ========================
def preprocess_text_id(text):
    if pd.isna(text):
        return []
    # (2) Hapus tanda baca & simbol
    text = re.sub(r"[^a-zA-Z\s]", " ", text.lower())
    # (5) Tokenisasi
    tokens = text.split()
    if not tokens:
        return []
    # (1) Stopword removal
    tokens = [w for w in tokens if w not in stopwords_id]
    if not tokens:
        return []
    # (4) Stemming dengan Sastrawi
    tokens = [stemmer_id.stem(w) for w in tokens]
    return tokens

# ========================
# Fungsi Preprocessing English
# ========================
def preprocess_text_en(text):
    if pd.isna(text):
        return []
    # (2) Hapus tanda baca & simbol
    text = re.sub(r"[^a-zA-Z\s]", " ", text.lower())
    # (5) Tokenisasi
    tokens = text.split()
    if not tokens:
        return []
    # (3) Cek ejaan pembakuan kata (hindari None)
    corrected = []
    for w in tokens:
        corr = spell_en.correction(w)
        corrected.append(corr if corr is not None else w)
    tokens = corrected
    # (1) Stopword removal
    tokens = [w for w in tokens if isinstance(w, str) and w not in stopwords_en]
    if not tokens:
        return []
    # (4) Lematisasi dengan spaCy (hanya jika ada token)
    doc = nlp(" ".join(tokens))
    tokens = [token.lemma_ for token in doc]
    return tokens

# === Terapkan ke abstrak Fakultas Teknik ===
pta_teknik["abstrak_id_clean"] = pta_teknik["abstrak_id"].apply(preprocess_text_id)
pta_teknik["abstrak_en_clean"] = pta_teknik["abstrak_en"].apply(preprocess_text_en)

# === Simpan hasil ===
pta_teknik.to_csv("preprocessing_pta_teknik.csv", index=False)

# Contoh hasil
print("Jumlah data Fakultas Teknik:", len(pta_teknik))
display(pta_teknik[["prodi", "abstrak_id_clean", "abstrak_en_clean"]].head())

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


### Pre-Pocessing Berita

In [24]:
import pandas as pd
import re
import nltk
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.corpus import stopwords

# Download stopwords (sekali saja)
nltk.download('stopwords')

# === Load dataset ===
tempo_berita = pd.read_csv("tempo_berita.csv")

# === Stopwords & Stemmer Indonesia ===
stopwords_id = set(stopwords.words("indonesian"))
factory = StemmerFactory()
stemmer_id = factory.create_stemmer()

# ========================
# Fungsi Preprocessing Indo
# ========================
def preprocess_text_id(text):
    if pd.isna(text):
        return ""
    # Menghilangkan simbol & tanda baca
    text = re.sub(r"[^a-zA-Z\s]", " ", text.lower())
    # Tokenisasi
    tokens = text.split()
    # Stopword removal
    tokens = [w for w in tokens if w not in stopwords_id]
    # Stemming dengan Sastrawi
    tokens = [stemmer_id.stem(w) for w in tokens]
    return tokens

# === Terapkan ke Tempo ===
tempo_berita["judul_clean"] = tempo_berita["judul_berita"].apply(preprocess_text_id)
tempo_berita["isi_clean"] = tempo_berita["isi_berita"].apply(preprocess_text_id)

# === Simpan hasil ===
tempo_berita.to_csv("preprocessing_berita.csv", index=False)

# Contoh hasil
tempo_berita[["judul_clean", "isi_clean"]].head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,judul_clean,isi_clean
0,"[ruu, ampas, aset, upaya, akomodasi, pidana, k...","[ketua, badan, legislasi, dpr, bob, hasan, dra..."
1,"[debat, sengit, komisi, ii, dpr, baleg, rebut,...","[suasana, rapat, koordinasi, pimpin, komisi, k..."
2,"[sengketa, pilkada, ulang, barito, utara, papu...","[mahkamah, konstitusi, mk, selesai, mohon, sel..."
3,"[respons, perintah, tentara, mk, tolak, gugat,...","[wakil, menteri, hukum, eddy, hiariej, hormat,..."
4,"[rangkap, jabat, menpora, tum, pssi, erick, tu...","[menteri, pemuda, olahraga, erick, thohir, nas..."
